In [1]:
import sys
import time
import threading
import multiprocessing

sys.path.append("../")
from IoTPy.core.stream import Stream, StreamArray, run
from IoTPy.agent_types.op import map_element, map_list, map_window
from IoTPy.helper_functions.recent_values import recent_values
from IoTPy.helper_functions.print_stream import print_stream

from IoTPy.concurrency.multicore import get_processes, get_processes_and_procs
from IoTPy.concurrency.multicore import terminate_stream
from IoTPy.concurrency.multicore import get_proc_that_inputs_source
from IoTPy.concurrency.multicore import extend_stream


## Actuators
A multiprocessing block may need to interact asynchronously with some external device. To do so, the block puts data into a queue and uses threads responsible for interfacing between the queue and the device. This simple example illustrates the simplest actuator: a printer. Indeed printing can be done synchronously by the multiprocessing block. Printing doesn't need a queue to interface between it and the block. We use the printer in this example to illustrate the idea.
<br>
<br>
Function <i>g</i> of process <i>p1</i> has an agent called 'copy_stream_s_to_queue_q' which  copies stream <i>s</i> to queue <i>q</i>. A thread, <b>my_thread</b> in <i>p1</i> prints values from the queue; this thread represents the thread that interfaces with an external actuator device. This thread is in addition to any source threads that may exist.
<br>
<br>
Queue <i>q</i> is specified as an <b>output queue</b>. An output queue gets a special message <b>'_finished'</b> when the multiprocess block terminates.
<br>
<br>
Threads (apart from source threads) and output queues are specified in <i>multicore_specifications</i>. See
<br>
<br>
{'name': 'p1', 'agent': g, 'inputs': ['y'], 
<br>
 'args': [q], <b>'output_queues'</b>: [q], <b>'threads'</b>: [my_thread]}
<br>
<br>
The thread, <i>my_thread</i>, terminates when it receives a '_finished' message. We want this thread to terminate so that process <i>p1</i> terminates, and then the entire multiprocessing block can terminate as well.

In [2]:
import threading
from IoTPy.agent_types.sink import stream_to_queue

def f(in_streams, out_streams):
    map_element(lambda v: v+100, in_streams[0], out_streams[0])

def source_thread_target(procs):
    for i in range(3):
        extend_stream(procs, data=list(range(i*2, (i+1)*2)), stream_name='x')
        time.sleep(0.001)
    terminate_stream(procs, stream_name='x')

def example_output_thread_with_queue():
    q = multiprocessing.Queue()

    def g(in_streams, out_streams, q):
        s = Stream('s')
        map_element(lambda v: v*2, in_streams[0], s)
        stream_to_queue(s, q, name='copy_stream_s_to_queue_q')

    def get_data_from_output_queue(q):
        while True:
            v = q.get()
            if v == '_finished': break
            else: print ('q.get() = ', v)

    multicore_specification = [
        # Streams
        [('x', 'i'), ('y', 'i')],
        # Processes
        [{'name': 'p0', 'agent': f, 'inputs':['x'], 'outputs': ['y'], 'sources': ['x']},
         {'name': 'p1', 'agent': g, 'inputs': ['y'], 
          'args': [q], 'output_queues': [q]}]
    ]

    processes, procs = get_processes_and_procs(multicore_specification)
    source_thread = threading.Thread(target=source_thread_target, args=(procs,))
    output_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))
    procs['p0'].threads = [source_thread, output_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_output_thread_with_queue()

q.get() =  200
q.get() =  202
q.get() =  204
q.get() =  206
q.get() =  208
q.get() =  210


## Example of Process Structure with Feedback
The example shows a process structure with feedback. This example creates an echo from a spoken sound. (You can write more efficient and succinct code to compute echoes. The code in this example is here merely because it illustrates a concept.)
<br>
### Streams
<ol>
    <li><b>sound_made</b>: This is the sound made by a speaker in a large spherical space.</li>
    <li><b>attenuated</b>: This is the sound made multiplied by an attenuation factor.</li>
    <li><b>echo</b>: This is the echo of the sound made heard at the center of the room. The echo is a delay followed by an attenuation of the sound heard. </li>
    <li><b>sound_heard</b>: This is the sound that is heard by the speaker. The heard sound is the sound made by the speaker plus the echo.</li>
</ol>
The equations that define the streams are:
<ol>
    <li>
    <b>attentuated[n] = sound_heard[n]*attenuation</b>
    </li>
    <li>
    <b>echo[n] = attentuated[n-delay]</b> for n > delay.
    </li>
    <li>
    <b>sound_heard[n] = sound_made[n] + echo[n]</b> for n > delay.
    </li>
</ol>

### Process Structure
Process <i>p0</i> has a source which feeds one of its input streams <i>sound_made</i> with a stream of measurements obtained from a microphone. In this example, the stream is generated with numbers so that we can see how streams are processed.
<br>
<br>
Process <i>p1</i> contains a single input stream which is the sound heard and a single output stream which is an attenuation of the sound heard. 

### Process Functions
The function <i>f</i> of <i>p0</i> computes <i>echo</i> from <i>sound_made</i>. The first 4 , i.e., <b>delay</b>, units of the echo are empty (i.e. 0). 
<br>
<b>map_element(lambda v: v, attenuated, echo)</b>
<br>
copies the attenuated stream to the echo stream; however, since the first 4 (i.e. delay) values of the echo stream are 0, the echo stream will consist of 4 zeroes followed by the attenuated stream.
<br>
<i>out_streams[0]</i> of process <i>p0</i> is <i>sound_heard</i>. Function <i>f</i> makes <i>sound_heard</i> the sum of the echo and the sound made.
<br>
The function <i>g</i> of process <i>p1</i> <i>p0</i> puts elements of its input stream (i.e. <i>sound_heard</i> on queue <i>q</i> and returns the elements multiplied by <i>attenuation</i>.

In [3]:
from IoTPy.agent_types.basics import *

def example_echo_two_cores():
    # This is the delay from when the made sound hits a
    # reflecting surface.
    delay = 4

    # This is the attenuation of the reflected wave.
    attenuation = 0.5

    # The results are put in this queue. A thread reads this
    # queue and feeds a speaker or headphone.
    q = multiprocessing.Queue()

    # Agent function for process named 'p0'
    # echo is a delay of zeroes followed by attenuated heard sound.
    # out_streams[0], which is the same as sound_heard is
    # echo + sound_made
    def f_echo(in_streams, out_streams, delay):
        sound_made, attenuated = in_streams
        echo = StreamArray('echo', dtype='float')
        echo.extend(np.zeros(delay, dtype='float'))
        map_element(lambda v: v, attenuated, echo)
        # The zip_map output is the sound heard which is
        # the sound heard plus the echo.
        zip_map(sum, [sound_made, echo], out_streams[0])

    # Agent function for process named 'p1'
    # This process puts the sound heard into the output queue
    # and returns an attenuated version of the sound_heard as 
    # its output stream.
    def g_echo(in_streams, out_streams, attenuation, q):
        def gg(v):
            # v is the sound heard
            q.put(v)
            # v*attenuation is the echo
            return v*attenuation
        map_element(gg, in_streams[0], out_streams[0])

    def source_thread_target(procs):
        data=np.arange(10, dtype='float')
        extend_stream(procs, data, stream_name='sound_made')
        time.sleep(0.0001)
        extend_stream(procs, data=np.zeros(10, dtype='float'), stream_name='sound_made')
        terminate_stream(procs, stream_name='sound_made')

    # Thread that gets data from the output queue
    # This thread is included in 'threads' in the specification.
    # Thread target
    def get_data_from_output_queue(q):
        finished_getting_output = False
        while not finished_getting_output:
            v = q.get()
            if v == '_finished': break
            print ('heard sound = spoken + echo: ', v)

    multicore_specification = [
        # Streams
        [('sound_made', 'f'), ('attenuated', 'f'), ('sound_heard', 'f')],
        # Processes
        [{'name': 'p0', 'agent': f_echo, 'inputs': ['sound_made', 'attenuated'], 
          'outputs': ['sound_heard'], 'keyword_args' : {'delay' : delay}, 'sources': ['sound_made']},
         {'name': 'p1', 'agent': g_echo, 'inputs': ['sound_heard'], 'outputs': ['attenuated'],
          'args': [attenuation, q], 'output_queues': [q] } ]]

    processes, procs = get_processes_and_procs(multicore_specification)
     
    source_thread = threading.Thread(target=source_thread_target, args=(procs,))
    output_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))
    procs['p0'].threads = [source_thread, output_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_echo_two_cores()

heard sound = spoken + echo:  0.0
heard sound = spoken + echo:  1.0
heard sound = spoken + echo:  2.0
heard sound = spoken + echo:  3.0
heard sound = spoken + echo:  4.0
heard sound = spoken + echo:  5.5
heard sound = spoken + echo:  7.0
heard sound = spoken + echo:  8.5
heard sound = spoken + echo:  10.0
heard sound = spoken + echo:  11.75
heard sound = spoken + echo:  3.5
heard sound = spoken + echo:  4.25
heard sound = spoken + echo:  5.0
heard sound = spoken + echo:  5.875
heard sound = spoken + echo:  1.75
heard sound = spoken + echo:  2.125
heard sound = spoken + echo:  2.5
heard sound = spoken + echo:  2.9375
heard sound = spoken + echo:  0.875
heard sound = spoken + echo:  1.0625


## Example source and actuator thread with single process
This example is the same as the previous one except that the computation is carried out in a single process rather in two processes. The example illustrates an actuator thread and a source thread in the same process.

In [4]:
def example_echo_single_core():
    # This is the delay from when the made sound hits a
    # reflecting surface.
    delay = 4

    # This is the attenuation of the reflected wave.
    attenuation = 0.5

    # The results are put in this queue. A thread reads this
    # queue and feeds a speaker or headphone.
    q = multiprocessing.Queue()

    # Agent function for process named 'p0'
    # echo is a delay of zeroes followed by attenuated heard sound.
    # out_streams[0], which is the same as sound_heard is
    # echo + sound_made
    def f_echo(in_streams, out_streams, delay, attenuation, q):
        echo = StreamArray(
            'echo', initial_value=np.array([0.0]*delay, dtype='float'), dtype='float')
        #Note: sound_made = in_streams[0]
        sound_heard = in_streams[0] + echo
        map_element(lambda v: v*attenuation, sound_heard, echo)
        stream_to_queue(sound_heard, q)

    def source_thread_target(procs):
        extend_stream(procs, data=np.arange(10, dtype='float'), stream_name='sound_made')
        time.sleep(0.0001)
        extend_stream(procs=procs, data=np.zeros(10, dtype='float'), stream_name='sound_made')
        terminate_stream(procs, stream_name='sound_made')

    # Thread that gets data from the output queue
    # This thread is included in 'threads' in the specification.
    # Thread target
    def get_data_from_output_queue(q):
        finished_getting_output = False
        while not finished_getting_output:
            v = q.get()
            if v == '_finished': break
            print ('heard sound = spoken + echo: ', v)

    multicore_specification = [
        # Streams
        [('sound_made', 'f')],
        # Processes
        [{'name': 'p0', 'agent': f_echo, 'inputs': ['sound_made'],
          'args' : [delay, attenuation, q], 'sources': ['sound_made'],'output_queues': [q]}]]

    processes, procs = get_processes_and_procs(multicore_specification)
     
    source_thread = threading.Thread(target=source_thread_target, args=(procs,))
    output_thread = threading.Thread(target=get_data_from_output_queue, args=(q,))
    procs['p0'].threads = [source_thread, output_thread]

    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

example_echo_single_core()

heard sound = spoken + echo:  0.0
heard sound = spoken + echo:  1.0
heard sound = spoken + echo:  2.0
heard sound = spoken + echo:  3.0
heard sound = spoken + echo:  4.0
heard sound = spoken + echo:  5.5
heard sound = spoken + echo:  7.0
heard sound = spoken + echo:  8.5
heard sound = spoken + echo:  10.0
heard sound = spoken + echo:  11.75
heard sound = spoken + echo:  3.5
heard sound = spoken + echo:  4.25
heard sound = spoken + echo:  5.0
heard sound = spoken + echo:  5.875
heard sound = spoken + echo:  1.75
heard sound = spoken + echo:  2.125
heard sound = spoken + echo:  2.5
heard sound = spoken + echo:  2.9375
heard sound = spoken + echo:  0.875
heard sound = spoken + echo:  1.0625


## Example of a grid computation
Grid computations are used in science, for example in computing the temperature of a metal plate. The grid is partitioned into regions with a process assigned to simulate each region. On the n-th step, each process reads the values of relevant parts of the grid and updates its own value.
<br>
<br>
This example uses two copies of the grid; the two copies are <b>even</b> and <b>odd</b>. On even steps (i.e., steps 0, 2, 4,..) each process reads the <i>even</i> grid and updates its portion of the <i>odd</i> grid. So, each portion of the grid is modified by only one process. And no process is reading values while the values are being modified.
<br>
By symmetry, on odd steps, each process reads the <i>odd</i> grid and updates its portion of the <i>even</i> grid.
<br>
### The example problem
A linear metal bar of length <i>N</i> is partitioned into a grid of <i>N</i> continuous regions. Grid 0 is kept at a constant temperature of 0 degrees while grid <i>N-1</i> is kept at a constant temperature of <i>N-1</i> degrees. Initially, the temperature at intermediate grid points is arbitrary; in the code below, the temperature at grid point <i>i</i> exceeds <i>i</i> by <i>DELTA</i>.
<br>
Let <b>TEMP[i][k]</b> be the temperature of the <i>i</i>-th region on step <i>k</i>. Then:
<ol>
    <li>TEMP[0][k] = 0 </li>
    <li>TEMP[N-1][k] = N-1 </li>
    <li>TEMP[i][k] = (TEMP[i-1][k] + TEMP[i][k] + TEMP[i+1][k])/3 i in [1, ..,N-2] </li>
</ol>


### Processes
The computation uses <i>N-2</i> processes. The <i>i</i>-th process is called 'grid_i' and is responsible for simulating the <i>i</i>-th region.
<br>
Each process takes the <i>k + 1</i>-th step after it has finished the <i>k</i>-th step and it has determined that its neighbors have also finished the <i>k</i>-th step.
<br>


### Streams
The system has one stream, <b>s_i</b> for the <i>i</i>-th process. This stream contains the elements [0, 1, .. , k] after the <i>i</i>-th process has completed <i>k</i>-th steps.
<br>
Process <i>grid_i</i> outputs stream <i>s_i</i> and inputs streams from its neighboring processes which are <i>grid_(i-1)</i> if <i>i</i> exceeds 1 and <i>grid_(i+1)</i> if <i>i</i> is less than <i>N-1</i>.

### Process Structure
The process structure is linear with each process getting input streams from each of its neighbors and sending its output stream to all its neighbors.

### Process Function
The process begins by sending 0 on its output stream to indicate that it has finished its 0-th step.
<br>
<br>
The <i>k</i>-th value of <i>in_streams[j]</i> is <i>k</i> when the <i>j</i>-th neighboring process has completed its <i>k</i>-th step.
<br>
<br>
<b>synch_stream</b> is an internal stream of the process. The <i>k</i>-th element of this stream is <i>k</i> after all neighboring processes have completed their <i>k</i>-th step.
<br>
<br>
The zip_map function <i>r</i> operates on a list with one element from each neighbor. All the elements of the list will be <i>k</i> on the <i>k</i>-th step. The zip_map function returns <i>k</i> which is any element of the list. In this example it returns the 0-th element.
<br>
<br>
Thus the zip_map function acts as a synchronizer. It waits until all neighbors have completed the <i>k</i>-step and then it outputs <i>k</i>.
<br>
<br>
Function <i>g</i> is called for the <i>k</i>-th time when this process and all its neighbors have completed <i>k - 1</i> steps. Function <i>g</i> does the grid computation. Function <i>r</i> and the zip_map agent are used merely for synchronizing.
### run()
Function <i>f</i> calls <b>run</b> after it has declared all its agents. Without calling run() the function will take no action.
<br>
<br>
Note that when using external source threads, you should not call <i>run</i> because the source threads are responsible for starting and stopping the main computational thread. This example has no source threads so you must call <i>run</i> to start the system.

In [5]:
from IoTPy.core.stream import _no_value
def test_grid():
    # N is the size of the grid
    N = 5
    # M is the number of steps of execution.
    M = 5
    # DELTA is the deviation from the final solution.
    DELTA = 0.01
    # even, odd are the grids that will be returned
    # by this computation
    even = multiprocessing.Array('f', N)
    odd = multiprocessing.Array('f', N)
    # Set up initial values of the grid.
    for i in range(1, N-1):
        even[i] = i + DELTA
    even[N-1] = N-1
    odd[N-1] = N-1
        
    def f(in_streams, out_streams, index, even, odd):
        def g(v):
            if (0 < index) and (index < N-1):
                if v%2 == 0:
                    odd[index] = (even[index-1] + even[index] + even[index+1])/3.0
                else:
                    even[index] = (odd[index-1] + odd[index] + odd[index+1])/3.0
            return v+1

        def r(lst, state):
            if state < M:
                return lst[0], state+1
            else:
                return _no_value, state
        for out_stream in out_streams: out_stream.extend([0])
        synch_stream = Stream('synch_stream')
        zip_map(r, in_streams, synch_stream, state=0, name='zip_map_'+str(index))
        map_element(g, synch_stream, out_streams[0], name='grid'+str(index))
        run()

    multicore_specification = [
        # Streams
        [('s_'+str(index), 'i') for index in range(1, N-1)],
        # Processes
        [{'name': 'grid_'+str(index), 'agent': f, 
          'inputs':['s_'+str(index+1), 's_'+str(index-1)], 
          'outputs':['s_'+str(index)], 
          'args': [index, even, odd]} for index in range(2, N-2)] + \
        [{'name': 'grid_'+str(1), 'agent': f, 
          'inputs':['s_'+str(2)], 'outputs':['s_'+str(1)], 
          'args': [1, even, odd]}] + \
        [{'name': 'grid_'+str(N-2), 'agent': f, 
          'inputs':['s_'+str(N-3)], 'outputs':['s_'+str(N-2)], 
          'args': [N-2, even, odd]}]
    ]

    # Execute processes (after including your own non IoTPy processes)
    processes = get_processes(multicore_specification)
    for process in processes: process.start()
    for process in processes: process.join()
    for process in processes: process.terminate()

    print ('Grid after ', M, ' steps is: ')
    if M%2 == 0:
        print (even[:])
    else:
        print (odd[:])

test_grid()

Grid after  5  steps is: 
[0.0, 1.002880573272705, 2.0040740966796875, 3.002880573272705, 4.0]
